In [8]:
import pandas as pd
import requests as rq
import os
import numpy as np
from bs4 import BeautifulSoup

O projeto usará como base os seguintes sites:
MLSSoccer - https://www.mlssoccer.com
Eurosport - https://www.eurosport.com
Soccerway - https://us.soccerway.com
FCTable - https://www.fctables.com
WhoScored - https://www.whoscored.com
FIFA - https://www.fifa.com
TopDrawerSoccer - https://www.topdrawersoccer.com
FoxSports - https://www.foxsports.com

In [9]:
sites = [('MLSSoccer','https://www.mlssoccer.com'),
('Eurosport','https://www.eurosport.com'),
('Soccerway','https://us.soccerway.com'),
('FCTable','https://www.fctables.com'),
('FIFA','https://www.fifa.com'),
('Fbref','https://fbref.com'),
('UEFA','https://www.uefa.com')]
##Duvidas:
#Eurosport vai ter como root https://www.eurosport.com/ ou https://www.eurosport.com/football ?
#Foxsports vai ter como root https://www.foxsports.com/ ou https://www.foxsports.com/soccer ?

In [10]:
#Robots.txt de cada site
#for i in range(len(sites)):
# urlRobots = sites[i][1] + '/robots.txt'
# headers = {'User-Agent': 'bvcl'}
# req = rq.get(urlRobots, headers=headers)
# directoryFile = 'robots/' + sites[i][0] + ".txt"
# with open(directoryFile, 'wb') as handle:
#    for block in req.iter_content():
#        handle.write(block)

In [11]:
#Pagina inicial de cada site
# for i in range(len(sites)):
#    urlHomePage = sites[i][1]
#    req = rq.get(urlHomePage)
#    directoryFile = 'pages/' + sites[i][0] + "/homePage.html" 
#    os.makedirs(os.path.dirname(directoryFile), exist_ok=True)
#    with open(directoryFile, 'wb') as handle:
#        for block in req.iter_content():
#            handle.write(block)

In [12]:
# urlHomePage = sites[6][1]
# req = rq.get(urlHomePage)
# parsedPage = BeautifulSoup(req.text,"html.parser")
# anchors = parsedPage.find_all('a', href = True)
# anchors

In [13]:
#Construindo array de allow e disallow do robots.txt
###Duvida: Levar em consideração Visit-time e Crawl-delay do robots ?
def criarDisallowArray(pageCounter):
    robotsTXT = 'robots/' + sites[pageCounter][0] + ".txt"
    with open(robotsTXT) as f:
       content = f.readlines()

    start = len(content)
    end = len(content)
    disallowArray = []
    allowArray = []
    visitTime = ''
    crawlDelay = ''
    for i in range(len(content)):
       if('User-agent: *' in content[i]):
           start = i
           break
    for i in range(start+1, len(content)):
       if('User-agent: ' in content[i]):
           end = i
           break
    for i in range(start+1, len(content)):
       if('Visit-time: ' in content[i]):
           visitTime = content[i].split(' ')[1]
           break
    for i in range(start+1, len(content)):
       if('Crawl-delay: ' in content[i]):
           crawlDelay = content[i].split(' ')[1].split('\n')[0]
           break
    for i in range (start+1, end-1):
       line = content[i].split(' ')
       if(line[0] == 'Allow:'):
           allowArray.append(line[1].split('\n')[0])
       elif(line[0] == 'Disallow:'):
           disallowArray.append(line[1].split('\n')[0])
    return disallowArray

In [14]:
# for j in range(len(anchors)):
#     for i in range (len(disallowArray)):
#         if(disallowArray[i] in anchors[j]['href']):
#             print(anchors[j]['href'])
# for j in range(20):
#     print(anchors[j]['href'])

1 - Abordagem BFS

In [15]:
#Funções auxiliares 
def isAllowed(disallowArr,url):
    for i in range (len(disallowArr)):
        if(disallowArr[i] in url):
            return False
    return True

In [16]:
def isRootSite(siteUrl):
    return not('http://' in siteUrl or 'https://' in siteUrl)

In [17]:
def getLinks(page):
    parsedPage = BeautifulSoup(page,"html.parser")
    anchors = parsedPage.find_all('a', href = True)
    links = []
    for anchor in anchors:
        links.append(anchor['href'])
    return links

In [18]:
def getPage(urlPage):
    req = rq.get(urlPage)
    return req.text

In [19]:
def downloadPage(urlPage,siteFolder,pageCounter):
    try:
        req = rq.get(urlPage)
        directoryFile = 'pages/' + siteFolder + "/page" + str(pageCounter) + ".html" 
        os.makedirs(os.path.dirname(directoryFile), exist_ok=True)
        with open(directoryFile, 'wb') as handle:
           for block in req.iter_content():
               handle.write(block)
        return req.text
    except rq.exceptions.Timeout:
        print("Timeout occurred")
        

In [20]:
# urlHomePage = sites[0][1]
# isAllowed(disallowArray,'/account/me')
# isRootSite('/league/history')
# p = getPage(urlHomePage)
# getLinks(p)

In [32]:
import queue
#for i in range (7,len(sites)):
i=6
urlQueue = queue.Queue()
visited = set()
downloaded = []
disallowArrayToUse = criarDisallowArray(i)
siteFolder = sites[i][0]
siteRootUrl = sites[i][1]
visited.add(siteRootUrl)
urlQueue.put(siteRootUrl)
pageCounter = 0;
while((not urlQueue.empty()) and pageCounter<999): 
    currentUrl = urlQueue.get()
    print(currentUrl)
    page = downloadPage(currentUrl,siteFolder,pageCounter)
    downloaded.append(currentUrl)
    pageCounter+=1
    #print(pageCounter)
    for url in getLinks(page):
        if ((siteRootUrl+url not in visited) and isRootSite(url) and isAllowed(disallowArrayToUse,url) and (url.startswith('/')) and ('{' not in url) and (':' not in url)):
            visited.add(siteRootUrl+url)
            urlQueue.put(siteRootUrl+url)

#downloaded

https://www.uefa.com
https://www.uefa.com/
https://www.uefa.com/insideuefa/
https://www.uefa.com/livescores/
https://www.uefa.com/tickets/
https://www.uefa.com/uefachampionsleague/
https://www.uefa.com/uefaeuropaleague/
https://www.uefa.com/uefaeuro-2020/
https://www.uefa.com/insideuefa/member-associations/
https://www.uefa.com/women/
https://www.uefa.com/memberassociations/leaguesandcups/
https://www.uefa.com/tvguide/
https://www.uefa.com/uefasupercup/
https://www.uefa.com/uefayouthleague/
https://www.uefa.com/european-qualifiers/
https://www.uefa.com/uefanationsleague/
https://www.uefa.com/under21/
https://www.uefa.com/under19/
https://www.uefa.com/under17/
https://www.uefa.com/regionscup/
https://www.uefa.com/womenseuro/
https://www.uefa.com/womensworldcup/
https://www.uefa.com/womenschampionsleague/
https://www.uefa.com/womensunder19/
https://www.uefa.com/womensunder17/
https://www.uefa.com/futsaleuro/
https://www.uefa.com/futsalworldcup/
https://www.uefa.com/uefafutsalchampionslea

https://www.uefa.com/uefachampionsleague/matches/
https://www.uefa.com/uefachampionsleague/standings/
https://www.uefa.com/uefachampionsleague/draws/
https://www.uefa.com/uefachampionsleague/clubs/
https://www.uefa.com/uefachampionsleague/statistics/
https://www.uefa.com/uefachampionsleague/history/
https://www.uefa.com/uefachampionsleague/about/
https://www.uefa.com/uefachampionsleague/season=2020/matches/round=2001140/match=2027034/
https://www.uefa.com/uefachampionsleague/season=2020/matches/round=2001140/match=2027037/
https://www.uefa.com/uefachampionsleague/season=2020/matches/round=2001140/match=2027033/
https://www.uefa.com/uefachampionsleague/season=2020/matches/round=2001140/match=2027032/
https://www.uefa.com/uefachampionsleague/season=2020/matches/round=2001140/match=2027035/
https://www.uefa.com/uefachampionsleague/season=2020/matches/round=2001140/match=2027036/
https://www.uefa.com/uefachampionsleague/season=2020/matches/round=2001140/match=2027039/
https://www.uefa.com/

https://www.uefa.com/uefaeuropaleague/season=2020/matches/round=2001150/match=2027794/
https://www.uefa.com/uefaeuropaleague/season=2020/matches/round=2001150/match=2027793/
https://www.uefa.com/uefaeuropaleague/season=2020/matches/round=2001150/match=2027792/
https://www.uefa.com/uefaeuropaleague/season=2020/matches/round=2001150/match=2027791/
https://www.uefa.com/uefaeuropaleague/season=2020/matches/round=2001150/match=2027790/
https://www.uefa.com/uefaeuropaleague/season=2020/matches/round=2001150/match=2027789/
https://www.uefa.com/uefaeuropaleague/season=2020/matches/
https://www.uefa.com/uefaeuropaleague/season=2020/clubs/club=50118/
https://www.uefa.com/uefaeuropaleague/season=2020/clubs/club=59028/
https://www.uefa.com/uefaeuropaleague/season=2020/clubs/club=60609/
https://www.uefa.com/uefaeuropaleague/season=2020/clubs/club=52714/
https://www.uefa.com/uefaeuropaleague/season=2020/clubs/club=52709/
https://www.uefa.com/uefaeuropaleague/season=2020/clubs/club=52723/
https://www

https://www.uefa.com/insideuefa/member-associations/association=cyp/
https://www.uefa.com/insideuefa/member-associations/association=cze/
https://www.uefa.com/insideuefa/member-associations/association=den/
https://www.uefa.com/insideuefa/member-associations/association=eng/
https://www.uefa.com/insideuefa/member-associations/association=est/
https://www.uefa.com/insideuefa/member-associations/association=fro/
https://www.uefa.com/insideuefa/member-associations/association=fin/
https://www.uefa.com/insideuefa/member-associations/association=fra/
https://www.uefa.com/insideuefa/member-associations/association=geo/
https://www.uefa.com/insideuefa/member-associations/association=ger/
https://www.uefa.com/insideuefa/member-associations/association=gib/
https://www.uefa.com/insideuefa/member-associations/association=gre/
https://www.uefa.com/insideuefa/member-associations/association=hun/
https://www.uefa.com/insideuefa/member-associations/association=isl/
https://www.uefa.com/insideuefa/me

https://www.uefa.com/memberassociations/association=eng/domesticleague/matches/index.html
https://www.uefa.com/teamsandplayers/teams/club=2604200/domestic/index.html
https://www.uefa.com/teamsandplayers/teams/club=50147/domestic/index.html
https://www.uefa.com/teamsandplayers/teams/club=50064/domestic/index.html
https://www.uefa.com/teamsandplayers/teams/club=52819/domestic/index.html
https://www.uefa.com/teamsandplayers/teams/club=50149/domestic/index.html
https://www.uefa.com/teamsandplayers/teams/club=2604307/domestic/index.html
https://www.uefa.com/teamsandplayers/teams/club=73927/domestic/index.html
https://www.uefa.com/memberassociations/association=por/domesticleague/standings/index.html
https://www.uefa.com/memberassociations/association=por/domesticleague/matches/index.html
https://www.uefa.com/teamsandplayers/teams/club=2601038/domestic/index.html
https://www.uefa.com/teamsandplayers/teams/club=52826/domestic/index.html
https://www.uefa.com/teamsandplayers/teams/club=64500/do

https://www.uefa.com/memberassociations/association=hun/domesticleague/index.html
https://www.uefa.com/memberassociations/association=isl/domesticleague/index.html
https://www.uefa.com/memberassociations/association=isr/domesticleague/index.html
https://www.uefa.com/memberassociations/association=ita/domesticleague/index.html
https://www.uefa.com/memberassociations/association=kaz/domesticleague/index.html
https://www.uefa.com/memberassociations/association=kos/domesticleague/index.html
https://www.uefa.com/memberassociations/association=lva/domesticleague/index.html
https://www.uefa.com/memberassociations/association=ltu/domesticleague/index.html
https://www.uefa.com/memberassociations/association=lux/domesticleague/index.html
https://www.uefa.com/memberassociations/association=mlt/domesticleague/index.html
https://www.uefa.com/memberassociations/association=mda/domesticleague/index.html
https://www.uefa.com/memberassociations/association=mne/domesticleague/index.html
https://www.uefa

https://www.uefa.com//de.uefa.com/uefayouthleague/
https://www.uefa.com//ru.uefa.com/uefayouthleague/
https://www.uefa.com//es.uefa.com/uefayouthleague/
https://www.uefa.com//it.uefa.com/uefayouthleague/
https://www.uefa.com//pt.uefa.com/uefayouthleague/
https://www.uefa.com/european-qualifiers/stories/
https://www.uefa.com/european-qualifiers/season=2020/matches/
https://www.uefa.com/european-qualifiers/standings/
https://www.uefa.com/european-qualifiers/season=2020/teams/
https://www.uefa.com/european-qualifiers/season=2020/statistics/
https://www.uefa.com/european-qualifiers/about/
https://www.uefa.com/european-qualifiers/season=2020/matches/round=2001086/match=2026216/
https://www.uefa.com/european-qualifiers/season=2020/matches/round=2001086/match=2026217/
https://www.uefa.com/european-qualifiers/season=2020/matches/round=2001086/match=2026218/
https://www.uefa.com/european-qualifiers/season=2020/matches/round=2001086/match=2026220/
https://www.uefa.com/european-qualifiers/season=

https://www.uefa.com/european-qualifiers/season=2020/teams/team=117/
https://www.uefa.com/european-qualifiers/season=2020/teams/team=50168/
https://www.uefa.com/european-qualifiers/season=2020/teams/team=66/
https://www.uefa.com/european-qualifiers/season=2020/teams/team=42/
https://www.uefa.com/european-qualifiers/season=2020/teams/team=57419/
https://www.uefa.com/european-qualifiers/season=2020/teams/team=60094/
https://www.uefa.com/european-qualifiers/season=2020/teams/team=49/
https://www.uefa.com/european-qualifiers/season=2020/teams/team=81/
https://www.uefa.com/european-qualifiers/season=2020/standings/
https://www.uefa.com/uefaeuro-2020/video/videoid=2623709.html
https://www.uefa.com/uefaeuro-2020/video/videoid=2619581.html
https://www.uefa.com/uefaeuro-2020/video/videoid=2391166.html
https://www.uefa.com/european-qualifiers/video/stars-of-european-qualifiers/videoid=2622868.html
https://www.uefa.com/european-qualifiers/video/stars-of-european-qualifiers/videoid=2622669.html
ht

https://www.uefa.com/uefanationsleague/season=2019/teams/team=57155/
https://www.uefa.com/uefanationsleague/season=2019/teams/team=82/
https://www.uefa.com/uefanationsleague/season=2019/teams/team=57160/
https://www.uefa.com/uefanationsleague/season=2019/teams/team=50168/
https://www.uefa.com/uefanationsleague/season=2019/teams/team=2608110/
https://www.uefa.com/uefanationsleague/season=2019/teams/team=57154/
https://www.uefa.com/uefanationsleague/season=2019/teams/team=51218/
https://www.uefa.com/uefanationsleague/season=2019/teams/team=88/
https://www.uefa.com/uefanationsleague/season=2019/teams/team=59205/
https://www.uefa.com/uefanationsleague/season=2019/teams/team=57419/
https://www.uefa.com/uefanationsleague/season=2019/teams/team=94080/
https://www.uefa.com/uefanationsleague/season=2019/teams/team=81/
https://www.uefa.com/uefanationsleague/season=2019/standings/
https://www.uefa.com/uefanationsleague/video/videoid=2609112.html
https://www.uefa.com/uefanationsleague/video/videoi

In [33]:
# pCount = 472
# siteFolder = sites[5][0]
# for n in urlQueue.queue:
#     page = downloadPage(n,siteFolder,pCount)
#     pCount+=1
#     print(pCount)
#     if(pCount>998):
#         break